In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from lenstronomy.Util.param_util import ellipticity2phi_q
import glob
from scipy.special import gamma

from scipy.special import gamma, gammainc
from scipy.optimize import brentq

In [2]:
def get_total_flux(kwargs):
    n_sersic = kwargs['n_sersic']
    e1 = kwargs['e1']
    e2 = kwargs['e2']
    amp = kwargs['amp']
    r_sersic = kwargs['R_sersic']
    
    phi, q = param_util.ellipticity2phi_q(e1, e2)
    
    b_n = 1.9992 * n_sersic - 0.3271
    flux = q * 2*np.pi * n_sersic * amp * r_sersic**2 * np.exp(b_n) * b_n**(-2*n_sersic) * gamma(2*n_sersic)
    
    return flux


def _flux(r, amp, n, r_s):
    bn = 1.9992 * n - 0.3271
    x = bn * (r/r_s)**(1./n)
    
    return amp * r_s**2 * 2 * np.pi * n * np.exp(bn) / bn**(2*n) * gammainc(2*n, x) * gamma(2*n)

def _total_flux(amp, n, r_s):
    bn = 1.9992 * n - 0.3271

    return amp * r_s**2 * 2 * np.pi * n * np.exp(bn) / bn**(2*n) * gamma(2*n)

def get_half_light_radius(amp0,n0,R0,amp1,n1,R1):
    tot_flux = _total_flux(amp0, n0, R0) + _total_flux(amp1, n1, R1)
    def func(r):
        return _flux(r, amp0, n0, R0) +  _flux(r, amp1, n1, R1) - tot_flux/2.
    return brentq(func, 0.01, 30)

In [3]:
def light_assigner_double(lens_name,survey,num_step=500,walker_ratio = 16):
    
    data_dict = {"lens_name":lens_name,"light_model":"double_sersic"}
    
    chain_list = np.load('../models/{}_double_chains.npy'.format(lens_name), allow_pickle=True)
    double_df = np.load('../models/{}_double_results.npy'.format(lens_name), allow_pickle=True)
    
    params_double = chain_list[0][2]
    num_params = len(params_double)
    num_walkers = num_params*walker_ratio
    chain = np.empty((num_walkers, num_step, num_params))
    for i in np.arange(num_params):
        samples = np.array(chain_list[1][1])[:, i].T
        chain[:, :, i] = samples.reshape((num_step, num_walkers)).T
            
    
    R0_std = np.std(chain[:, -1, params_double.index('R_sersic_lens_light0')])
    R1_std = np.std(chain[:, -1, params_double.index('R_sersic_lens_light1')])  

    reff_med = get_half_light_radius(double_df[0]['amp'],double_df[0]['n_sersic'],double_df[0]['R_sersic'],
                                     double_df[1]['amp'],double_df[1]['n_sersic'],double_df[1]['R_sersic'])
    reff_exte1 = get_half_light_radius(double_df[0]['amp'],double_df[0]['n_sersic'],double_df[0]['R_sersic']-R0_std,
                                     double_df[1]['amp'],double_df[1]['n_sersic'],double_df[1]['R_sersic']-R1_std)
    reff_exte2 = get_half_light_radius(double_df[0]['amp'],double_df[0]['n_sersic'],double_df[0]['R_sersic']+R0_std,
                                     double_df[1]['amp'],double_df[1]['n_sersic'],double_df[1]['R_sersic']+R1_std)
    
    reff = round(reff_med,3)
    reff_e1 = round(reff_med-reff_exte1,3)
    reff_e2 = round(reff_exte2-reff_med,3)
    
    if reff_e1<0:
        print("Negative error at lens : {}".format(lens_name))
        reff_e1 = - reff_e1
    if reff_e2<0:
        print("Negative error at lens : {}".format(lens_name))
        reff_e2 = - reff_e2
        
    data_dict['r_eff']  = reff
    data_dict['r_eff_e1'] = reff_e1
    data_dict['r_eff_e2'] = reff_e2
        
    data_dict['r_sersic_1'] = double_df[0]['R_sersic']
    data_dict['n_sersic_1'] =  double_df[0]['n_sersic']
    data_dict['amp_sersic_1'] =  double_df[0]['amp']
    data_dict['e1_sersic1'],data_dict['e2_sersic1'] = double_df[0]['e1'],double_df[0]['e2'] 
    
    data_dict['r_sersic_2'] = double_df[1]['R_sersic']
    data_dict['n_sersic_2'] =  double_df[1]['n_sersic']
    data_dict['amp_sersic_2'] =  double_df[1]['amp']
    data_dict['e1_sersic2'],data_dict['e2_sersic2'] = double_df[1]['e1'],double_df[1]['e2']     
    
    return data_dict
             


In [4]:

def light_assigner_single(lens_name,survey,num_step=500,walker_ratio = 16):        
    #Single list
    data_dict = {"lens_name":lens_name,"light_model":"single_sersic"}
    
    single_chain_list = np.load('../models/{}_single_chains.npy'.format(lens_name), allow_pickle=True)
    single_df = np.load('../models/{}_single_results.npy'.format(lens_name), allow_pickle=True)
    
    params_single = single_chain_list[0][2]
    num_params = len(params_single)
    num_walkers = num_params*walker_ratio
    chain_single = np.empty((num_walkers, num_step, num_params))
    for i in np.arange(num_params):
        samples = np.array(single_chain_list[1][1])[:, i].T
        chain_single[:, :, i] = samples.reshape((num_step, num_walkers)).T   
        
        
    reff = round(np.median(chain_single[:, -1, 0]),3)
    reff_e1 = round(np.percentile(chain_single[:, -1, 0], 50.) - np.percentile(chain_single[:, -1, 0], 16.),3)
    reff_e2 = round(np.percentile(chain_single[:, -1, 0], 84.) - np.percentile(chain_single[:, -1, 0], 50.),3)        
         
    
    data_dict['r_eff']  = reff
    data_dict['r_eff_e1'] = reff_e1
    data_dict['r_eff_e2'] = reff_e2
        
    data_dict['r_sersic_1'] = single_df[0]['R_sersic']
    data_dict['n_sersic_1'] =  single_df[0]['n_sersic']
    data_dict['amp_sersic_1'] =  single_df[0]['amp']
    data_dict['e1_sersic1'],data_dict['e2_sersic1'] = single_df[0]['e1'],single_df[0]['e2']     

    return data_dict

In [5]:
Double_SLACS = ['SDSSJ0029-0055', 'SDSSJ0037-0942', 'SDSSJ0819+4534', 'SDSSJ0903+4116', 'SDSSJ0936+0913',
                'SDSSJ0959+0410', 'SDSSJ1134+6027', 'SDSSJ1204+0358', 'SDSSJ1213+6708', 'SDSSJ1218+0830',
                'SDSSJ1531-0105', 'SDSSJ1621+3931', 'SDSSJ1627-0053', 'SDSSJ2302-0840']

Single_SLACS = ['SDSSJ0008-0004', 'SDSSJ0252+0039', 'SDSSJ0330-0020', 'SDSSJ0728+3835', 'SDSSJ0737+3216',
                'SDSSJ0912+0029', 'SDSSJ1023+4230', 'SDSSJ1100+5329', 'SDSSJ1112+0826', 'SDSSJ1250+0523',
                'SDSSJ1306+0600', 'SDSSJ1313+4615', 'SDSSJ1402+6321', 'SDSSJ1630+4520', 'SDSSJ1636+4707',
                'SDSSJ2238-0754', 'SDSSJ2300+0022', 'SDSSJ2303+1422', 'SDSSJ2343-0030', 'SDSSJ2347-0005']

Double_SL2S = ['SL2SJ0208-0714', 'SL2SJ0219-0829', 'SL2SJ1427+5516']
Single_SL2S = ['SL2SJ0214-0405', 'SL2SJ0217-0513', 'SL2SJ0225-0454', 'SL2SJ0226-0406', 'SL2SJ0226-0420',
               'SL2SJ0232-0408', 'SL2SJ0849-0251', 'SL2SJ0849-0412', 'SL2SJ0858-0143', 'SL2SJ0901-0259',
               'SL2SJ0904-0059', 'SL2SJ0959+0206', 'SL2SJ1358+5459', 'SL2SJ1359+5535', 'SL2SJ1401+5544',
               'SL2SJ1402+5505', 'SL2SJ1405+5243', 'SL2SJ1406+5226', 'SL2SJ1411+5651', 'SL2SJ1420+5630',
               'SL2SJ2214-1807']

Double_BELLS = ['SDSSJ0801+4727', 'SDSSJ0944-0147', 'SDSSJ1234-0241', 'SDSSJ1349+3612', 'SDSSJ1542+1629',
                'SDSSJ1631+1854', 'SDSSJ2125+0411']
Single_BELLS = ['SDSSJ0151+0049', 'SDSSJ0747+5055', 'SDSSJ0830+5116', 'SDSSJ1159-0007', 'SDSSJ1215+0047',
                'SDSSJ1221+3806', 'SDSSJ1318-0104', 'SDSSJ1337+3620', 'SDSSJ1352+3216', 'SDSSJ1545+2748',
                'SDSSJ1601+2138', 'SDSSJ2303+0037']


print(light_assigner_double('SDSSJ1627-0053','SLACS'))
print(light_assigner_single('SL2SJ1401+5544','SL2S'))


{'lens_name': 'SDSSJ1627-0053', 'light_model': 'double_sersic', 'r_eff': 2.981, 'r_eff_e1': 0.054, 'r_eff_e2': 0.055, 'r_sersic_1': 1.2359239259796988, 'n_sersic_1': 4.0, 'amp_sersic_1': 19.230466943666674, 'e1_sersic1': -0.09159640911733445, 'e2_sersic1': 0.016970777015809225, 'r_sersic_2': 6.865504102702223, 'n_sersic_2': 1.0, 'amp_sersic_2': 0.8364101556820546, 'e1_sersic2': -0.09159640911733445, 'e2_sersic2': 0.016970777015809225}
{'lens_name': 'SL2SJ1401+5544', 'light_model': 'single_sersic', 'r_eff': 1.637, 'r_eff_e1': 0.008, 'r_eff_e2': 0.01, 'r_sersic_1': 1.6387959029283512, 'n_sersic_1': 4.0, 'amp_sersic_1': 13.208840027166163, 'e1_sersic1': -0.07316667535966452, 'e2_sersic1': 0.0856956130883916}
